In [1]:
import sys
sys.path.insert(0, "/u/home/m/michaelw/project-ngarud/microbiome_evolution/microbiome_evolution_MOUSE/")

import sample_utils 
import config
import parse_midas_data
import os.path
import pylab
import numpy

import diversity_utils
import gene_diversity_utils

import stats_utils
from math import log10,ceil
from numpy.random import randint

import core_gene_utils
import gzip
import os


# Question 1: what is `sample_utils.calculate_subject_pairs()` not working

In [2]:
debug = False
chunk_size = 1000000000
species_name="Akkermansia_muciniphila_55290"
good_species_list = [species_name]

In [3]:
# Load subject and sample metadata
sys.stderr.write("Loading sample metadata...\n")
subject_sample_map = parse_midas_data.parse_subject_sample_map()

Loading sample metadata...


In [4]:
# header for the output file.
record_strs = [", ".join(['Species', 'Sample1', 'Sample2', 'Type', 'Num_muts', 'Num_revs', 'Num_mut_opportunities', 'Num_rev_opportunities'])]

### Henceforth, I will enter the `for species in good_species_list:` for loop, as we know it's *Akkermansia_muciniphila_55290*

In [5]:
sys.stderr.write("Loading haploid samples...\n")

# Only plot samples above a certain depth threshold that are confidently phaseable.
snp_samples = diversity_utils.calculate_haploid_samples(species_name, debug=debug)

Loading haploid samples...


In [6]:
snp_samples

array(['M8CeCGG', 'M8JGG_2', 'M8IGG_2', 'M7CecGG', 'M8ColonGG_2',
       'M7IGG_2', 'TL1gDNAshort', 'M7JGG', 'M7DGG', 'M8D', 'M7ColonGG',
       'M1JC_CKDN220050970-1A_H7MMHDSX5_L2',
       'M1CeC_CKDN220050952-1A_H7MMHDSX5_L3',
       'M1CoC_CKDN220050958-1A_H7MMHDSX5_L2',
       'M1IC_CKDN220050976-1A_H7MMHDSX5_L2',
       'M1DC_CKDN220050964-1A_H7MMHDSX5_L2',
       'M2IC_CKDN220050977-1A_H7MMHDSX5_L2',
       'M3CeC_CKDN220050954-1A_H7MMHDSX5_L2',
       'M6IG_CKDN220050981-1A_HF3V5DSX5_L1',
       'M2CeC_CKDN220050953-1A_merge',
       'M2CoC_CKDN220050959-1A_H7MMHDSX5_L2',
       'M3CoC_CKDN220050960-1A_H7MMHDSX5_L2',
       'M6JG_CKDN220050975-1A_H7MMHDSX5_L2',
       'M6CeG_CKDN220050957-1A_H7MMGDSX5_L2',
       'M4CoG_CKDN220050961-1A_merge',
       'M3JC_CKDN220050972-1A_H7MMHDSX5_L2',
       'M6CoG_CKDN220050963-1A_H7MMHDSX5_L2',
       'M2JC_CKDN220050971-1A_H7MMHDSX5_L2',
       'M5CeG_CKDN220050956-1A_H7MMHDSX5_L2',
       'M5CoG_CKDN220050962-1A_merge',
       'M4CeG_CKD

In [7]:
sys.stderr.write("Proceeding with %d haploid samples!\n" % len(snp_samples))

sys.stderr.write("Loading core genes...\n")
core_genes = core_gene_utils.parse_core_genes(species_name)
non_shared_genes = core_gene_utils.parse_non_shared_reference_genes(species_name)
shared_pangenome_genes = core_gene_utils.parse_shared_genes(species_name)
sys.stderr.write("Done! Core genome consists of %d genes\n" % len(core_genes))
sys.stderr.write("%d shared genes and %d non-shared genes\n" % (len(shared_pangenome_genes), len(non_shared_genes)))

Proceeding with 39 haploid samples!
Loading core genes...
Done! Core genome consists of 1505 genes
476 shared genes and 2106 non-shared genes


In [8]:
# Load SNP information for species_name
sys.stderr.write("Loading SNPs for %s...\n" % species_name)
sys.stderr.write("(core genes only...)\n")
pi_matrix_syn = numpy.array([])
avg_pi_matrix_syn = numpy.array([])

syn_mut_difference_matrix = numpy.array([]) # 4d sites in core genes
syn_mut_opportunity_matrix = numpy.array([])
syn_rev_difference_matrix = numpy.array([]) # 4d sites in core genes
syn_rev_opportunity_matrix = numpy.array([])

non_mut_difference_matrix = numpy.array([]) # 1d sites in core genes
non_mut_opportunity_matrix = numpy.array([])
non_rev_difference_matrix = numpy.array([]) # 1d sites in core genes
non_rev_opportunity_matrix = numpy.array([])

core_mut_difference_matrix = numpy.array([]) # all sites in core genes
core_mut_opportunity_matrix = numpy.array([])
core_rev_difference_matrix = numpy.array([]) # all sites in core genes
core_rev_opportunity_matrix = numpy.array([])

snp_mut_difference_matrix = numpy.array([]) # all sites in all genes
snp_mut_opportunity_matrix = numpy.array([])
snp_rev_difference_matrix = numpy.array([]) # all sites in all genes
snp_rev_opportunity_matrix = numpy.array([])

Loading SNPs for Akkermansia_muciniphila_55290...
(core genes only...)


### Skipping the while loop, because it should exit after one loop (the ouput of the first line is -1)

In [9]:
final_line_number = 0
#while final_line_number >= 0:


In [10]:
dummy_samples, allele_counts_map, passed_sites_map, final_line_number = parse_midas_data.parse_snps(species_name, debug=debug, allowed_samples=snp_samples, chunk_size=chunk_size,allowed_genes=non_shared_genes, initial_line_number=final_line_number)





1k sites processed...
2k sites processed...
3k sites processed...


In [11]:
sys.stderr.write("Calculating matrix of snp differences...\n")
# Synonymous (4D)

chunk_syn_mut_difference_matrix, chunk_syn_rev_difference_matrix, chunk_syn_mut_opportunity_matrix, chunk_syn_rev_opportunity_matrix = diversity_utils.calculate_mutation_reversion_matrix(allele_counts_map, passed_sites_map, allowed_genes=core_genes, allowed_variant_types=set(['4D']))

Calculating matrix of snp differences...


In [12]:
# Nonsynonymous (1D) 
chunk_non_mut_difference_matrix, chunk_non_rev_difference_matrix, chunk_non_mut_opportunity_matrix, chunk_non_rev_opportunity_matrix = diversity_utils.calculate_mutation_reversion_matrix(allele_counts_map, passed_sites_map, allowed_genes=core_genes, allowed_variant_types=set(['1D']))


In [13]:
# Core (all)
chunk_core_mut_difference_matrix, chunk_core_rev_difference_matrix, chunk_core_mut_opportunity_matrix, chunk_core_rev_opportunity_matrix = diversity_utils.calculate_mutation_reversion_matrix(allele_counts_map, passed_sites_map, allowed_genes=core_genes)

In [14]:
# All
chunk_snp_mut_difference_matrix, chunk_snp_rev_difference_matrix, chunk_snp_mut_opportunity_matrix, chunk_snp_rev_opportunity_matrix = diversity_utils.calculate_mutation_reversion_matrix(allele_counts_map, passed_sites_map, allowed_genes=non_shared_genes)


In [15]:
if snp_mut_difference_matrix.shape[0]==0:
    snp_mut_difference_matrix = numpy.zeros_like(chunk_snp_mut_difference_matrix)*1.0
    snp_mut_opportunity_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    snp_rev_difference_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    snp_rev_opportunity_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0

    syn_mut_difference_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    syn_mut_opportunity_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    syn_rev_difference_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    syn_rev_opportunity_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0

    non_mut_difference_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    non_mut_opportunity_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    non_rev_difference_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    non_rev_opportunity_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0

    core_mut_difference_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    core_mut_opportunity_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    core_rev_difference_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0
    core_rev_opportunity_matrix = numpy.zeros_like(snp_mut_difference_matrix)*1.0

In [16]:
# Add syn
syn_mut_difference_matrix += chunk_syn_mut_difference_matrix
syn_mut_opportunity_matrix += chunk_syn_mut_opportunity_matrix
syn_rev_difference_matrix += chunk_syn_rev_difference_matrix
syn_rev_opportunity_matrix += chunk_syn_rev_opportunity_matrix

# Add non
non_mut_difference_matrix += chunk_non_mut_difference_matrix
non_mut_opportunity_matrix += chunk_non_mut_opportunity_matrix
non_rev_difference_matrix += chunk_non_rev_difference_matrix
non_rev_opportunity_matrix += chunk_non_rev_opportunity_matrix

# Add core
core_mut_difference_matrix += chunk_core_mut_difference_matrix
core_mut_opportunity_matrix += chunk_core_mut_opportunity_matrix
core_rev_difference_matrix += chunk_core_rev_difference_matrix
core_rev_opportunity_matrix += chunk_core_rev_opportunity_matrix

# Add all
snp_mut_difference_matrix += chunk_snp_mut_difference_matrix
snp_mut_opportunity_matrix += chunk_snp_mut_opportunity_matrix
snp_rev_difference_matrix += chunk_snp_rev_difference_matrix
snp_rev_opportunity_matrix += chunk_snp_rev_opportunity_matrix

snp_samples = dummy_samples

In [17]:
# Now calculate gene differences
# Load gene coverage information for species_name
sys.stderr.write("Loading pangenome data for %s...\n" % species_name)
gene_samples, gene_names, gene_presence_matrix, gene_depth_matrix, marker_coverages,     gene_reads_matrix = parse_midas_data.parse_pangenome_data(species_name,allowed_samples=snp_samples, disallowed_genes=shared_pangenome_genes)
sys.stderr.write("Done! Loaded %d genes\n" % len(gene_names))

Loading pangenome data for Akkermansia_muciniphila_55290...
Done! Loaded 2437 genes


In [18]:
gene_sample_list = list(gene_samples)
gene_sample_set = set(gene_samples)


In [19]:
# Calculate matrix of number of genes that differ
sys.stderr.write("Calculating matrix of gene differences...\n")

gene_gain_matrix, gene_loss_matrix, gene_opportunity_matrix = gene_diversity_utils.calculate_coverage_based_gene_hamming_matrix_gain_loss(gene_reads_matrix, gene_depth_matrix, marker_coverages)




Calculating matrix of gene differences...


In [20]:
min_coverage = config.min_median_coverage
good_marker_coverages = (marker_coverages>=min_coverage)

In [21]:
good_marker_coverages

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [22]:
gene_gain_matrix = gene_gain_matrix*good_marker_coverages[:,None]*good_marker_coverages[None,:]
gene_loss_matrix = gene_loss_matrix*good_marker_coverages[:,None]*good_marker_coverages[None,:]
gene_opportunity_matrix = gene_opportunity_matrix*good_marker_coverages[:,None]*good_marker_coverages[None,:]

In [23]:
# Add records to output
# Calculate which pairs of idxs belong to the same sample, which to the same subject
# and which to different subjects
same_sample_idxs, same_subject_idxs, diff_subject_idxs = sample_utils.calculate_subject_pairs(subject_sample_map, snp_samples)



### Addendum... what the hell is going on in `sample_utils.calculate_subject_pairs()`???

In [24]:
# def calculate_subject_pairs(subject_sample_map, sample_list=[]):

#     if len(sample_list)==0:
#         sample_list = list(sorted(flatten_samples(subject_sample_map).keys()))
    
#     new_sample_list = []
#     for sample in sample_list:
#         if sample.endswith('c'):
#             new_sample_list.append(sample[:-1])
#         else: 
#             new_sample_list.append(sample)
    
#     sample_list = new_sample_list
    
#     # invert subject sample map
#     sample_subject_map = {}
#     for subject in subject_sample_map.keys():
#         for sample in subject_sample_map[subject].keys():
#             sample_subject_map[sample] = subject
    
#     same_sample_idx_lower = []
#     same_sample_idx_upper = []
#     same_subject_idx_lower = []
#     same_subject_idx_upper = []
#     diff_subject_idx_lower = []
#     diff_subject_idx_upper = []
        
#     for i in xrange(0,len(sample_list)):
#         same_sample_idx_lower.append(i)
#         same_sample_idx_upper.append(i)
#         for j in xrange(0,i):
#             if sample_subject_map[sample_list[i]]==sample_subject_map[sample_list[j]]:
#                 same_subject_idx_lower.append(i)
#                 same_subject_idx_upper.append(j)
#             else: 
#                 diff_subject_idx_lower.append(i)
#                 diff_subject_idx_upper.append(j)
    
#     same_sample_idxs = (numpy.array(same_sample_idx_lower,dtype=numpy.int32), numpy.array(same_sample_idx_upper,dtype=numpy.int32))
    
#     same_subject_idxs = (numpy.array(same_subject_idx_lower,dtype=numpy.int32), numpy.array(same_subject_idx_upper,dtype=numpy.int32))
    
#     diff_subject_idxs = (numpy.array(diff_subject_idx_lower,dtype=numpy.int32), numpy.array(diff_subject_idx_upper,dtype=numpy.int32))
    
#     return same_sample_idxs, same_subject_idxs, diff_subject_idxs

In [25]:
subject_sample_map = subject_sample_map
sample_list = snp_samples

In [26]:
# invert subject sample map
sample_subject_map = {}
for subject in subject_sample_map.keys():
    for sample in subject_sample_map[subject].keys():
        accession = list(subject_sample_map[subject][sample])[0] #MW: added this because the accessions and sample_ids were not the same. 
        sample_subject_map[accession] = subject                  #Prior to this, code was sample_subject_map[sample] = subject instead of 
                                                                 # sample_subject_map[accession] = subject
            

In [27]:
sample_subject_map

{'M1CeC_CKDN220050952-1A_H7MMHDSX5_L3': '1',
 'M1CoC_CKDN220050958-1A_H7MMHDSX5_L2': '1',
 'M1DC_CKDN220050964-1A_H7MMHDSX5_L2': '1',
 'M1IC_CKDN220050976-1A_H7MMHDSX5_L2': '1',
 'M1JC_CKDN220050970-1A_H7MMHDSX5_L2': '1',
 'M2CeC_CKDN220050953-1A_merge': '2',
 'M2CoC_CKDN220050959-1A_H7MMHDSX5_L2': '2',
 'M2DC_CKDN220050965-1A_H7MMHDSX5_L2': '2',
 'M2IC_CKDN220050977-1A_H7MMHDSX5_L2': '2',
 'M2JC_CKDN220050971-1A_H7MMHDSX5_L2': '2',
 'M3CeC_CKDN220050954-1A_H7MMHDSX5_L2': '3',
 'M3CoC_CKDN220050960-1A_H7MMHDSX5_L2': '3',
 'M3DC_CKDN220050966-1A_H7MMHDSX5_L2': '3',
 'M3IC_CKDN220050978-1A_H7MMHDSX5_L2': '3',
 'M3JC_CKDN220050972-1A_H7MMHDSX5_L2': '3',
 'M4CeG_CKDN220050955-1A_H7MMHDSX5_L2': '4',
 'M4CoG_CKDN220050961-1A_merge': '4',
 'M4DG_CKDN220050967-1A_H7MMHDSX5_L3': '4',
 'M4IG_CKDN220050979-1A_H7MMHDSX5_L2': '4',
 'M4JG_CKDN220050973-1A_merge': '4',
 'M5CeG_CKDN220050956-1A_H7MMHDSX5_L2': '5',
 'M5CoG_CKDN220050962-1A_merge': '5',
 'M5DG_CKDN220050968-1A_merge': '5',
 'M5IG_CKDN22

In [ ]:
#The offending code:
for i in xrange(0,len(sample_list)):
    sample = sample_list[i]
    #         if sample.endswith('c'):                     #MW: I commented this out because it's specific to HMP data
    #             sample = sample[:-1]
    subject = sample_subject_map[sample]
    if not subject in subject_idx_map:
        subject_idx_map[subject] = i

In [43]:
subject_idx_map = {}

In [39]:
i = 0

In [40]:
sample = sample_list[i]

In [41]:
subject = sample_subject_map[sample]

In [44]:
if not subject in subject_idx_map:
    subject_idx_map[subject] = i

In [45]:
subject_idx_map

{'8': 0}

In [46]:
subject_idx_map = {}

for i in xrange(0,len(sample_list)):
    sample = sample_list[i]
#         if sample.endswith('c'):                     #MW: I commented this out because it's specific to HMP data
#             sample = sample[:-1]
    subject = sample_subject_map[sample]
    if not subject in subject_idx_map:
        subject_idx_map[subject] = i

In [47]:
subject_idx_map

{'0': 6, '1': 11, '2': 16, '3': 17, '4': 24, '5': 28, '6': 18, '7': 3, '8': 0}

In [34]:
def calculate_unique_samples(subject_sample_map, sample_list=[]):

    if len(sample_list)==0:
        sample_list = list(sorted(flatten_samples(subject_sample_map).keys()))
    
    # invert subject sample map
    sample_subject_map = {}
    for subject in subject_sample_map.keys():
        for sample in subject_sample_map[subject].keys():
            accession = list(subject_sample_map[subject][sample])[0] #MW: added this because the accessions and sample_ids are not the same. 
            sample_subject_map[accession] = subject                  #Prior to this, code was sample_subject_map[sample] = subject instead of 
                                                                     # sample_subject_map[accession] = subject
    
    subject_idx_map = {}
        
    for i in xrange(0,len(sample_list)):
        sample = sample_list[i]
#         if sample.endswith('c'):                     #MW: I commented this out because it's specific to HMP data
#             sample = sample[:-1]
        subject = sample_subject_map[sample]
        if not subject in subject_idx_map:
            subject_idx_map[subject] = i
            
    unique_idxs = numpy.zeros(len(sample_list),dtype=numpy.bool_)
    for i in subject_idx_map.values():
        unique_idxs[i]=True
    
    return unique_idxs
    

In [38]:
calculate_unique_samples(subject_sample_map, sample_list)

array([ True, False, False,  True, False, False,  True, False, False,
       False, False,  True, False, False, False, False,  True,  True,
        True, False, False, False, False, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False])